<a href="https://colab.research.google.com/github/jmhuer/quantum_dots/blob/main/MultispectralSensorTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training and Testing Multispectral-Sensing System for Multiplex Detection


**This notebook is a complementary file of M. Pirbhai and J. M. Huerta, "A Multispectral-Sensing System with"(in review). This includes a GridSeach procedure on how to optimize the hyperparameters 6 different ML models using 10-cross validation GridSearch prodcedure, and 4 distinct evaluation metrics**

The dataset used in this tutorial is collected using Adafruit AS7341 sensor, for more intruxtion on how to collect the data pelase refer to the paper *Method: B. Setup for testing sensor system*  

This tutorial was created using following main packages versions:



*   Scikit-learn==1.1.1
*   Pytorch==1.11.0
*   Scortch==0.11.1
*   Numpy==1.22.4
*   Pandas==1.4.2
*   Plotly==5.8.0



In [22]:
!pip install skorch
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Process data 


##Load from google drive

Below we have a tool to automatically fetch a google drive entry, given the google file ID:

In [23]:
import re
import subprocess

def download_gdrive(id, print_stout=True):
  coomand = 'gdown https://drive.google.com/uc?id={}'.format(id)
  returned_value = subprocess.run(coomand, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
  if print_stout: print(returned_value.stdout.decode("utf-8"))
  else: print("Download Complete")

data = '14mee8d0GDbwNzIprVSJoWdojGtqO_aTX' ##google drive id of excell 
download_gdrive(data)

Downloading...
From: https://drive.google.com/uc?id=14mee8d0GDbwNzIprVSJoWdojGtqO_aTX
To: /content/QDots data - Juan.xlsx
100%|██████████| 39.7k/39.7k [00:00<00:00, 40.9MB/s]



##Process Excell file

Next we can open the excell file using pandas

In [24]:
import pandas as pd

def open_excel(filename):
    excell = pd.ExcelFile(filename)
    excell.sheet_names
    df = excell.parse("Sheet1")
    df.columns = df.columns.map(str)
    df = df.dropna().reset_index(drop=True)
    return df

df = open_excel("/content/QDots data - Juan.xlsx")

df.head()

,Unnamed: 0,Time (min),μL,nM λ1,nM λ2,λ1 (nm),λ2 (nm),# emitters λ1 (x1010),# emitters λ2 (x1010),415,445,480,515,555,590,630,680
0,2021-04-12,0.0,0.0,0.0,0.0,620.0,560.0,0.0000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0
1,2021-04-12,0.0,30.0,50.0,0.0,620.0,560.0,90.3321,0.0,9.0,20.0,21.0,67.0,106.0,666.0,4320.0,24.0
2,2021-04-12,1.0,30.0,50.0,0.0,620.0,560.0,90.3321,0.0,9.0,21.0,22.0,69.0,107.0,675.0,4413.0,25.0
3,2021-04-12,2.0,30.0,50.0,0.0,620.0,560.0,90.3321,0.0,9.0,21.0,22.0,69.0,107.0,677.0,4439.0,25.0
4,2021-04-12,3.0,30.0,50.0,0.0,620.0,560.0,90.3321,0.0,9.0,21.0,22.0,69.0,106.0,678.0,4457.0,25.0


#Pre-processing + model defenitions before training

## Define models

We will be testing 6 different models. Below we define:



1.   Linear Regression (LR)
2.   Support Vector Machine Regression (SVMR)
3.   Random Forest Regression (RFR)
4.   Nueral Network, Univariate (NN)
5.   Nueral Network, Multivariate (NN)
6.   Guassian Process Regression (GPR)

We will use the following modules for grid search and cross validation:


In [25]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

##Linear Regression

In [26]:
from sklearn.linear_model import LinearRegression

lr_reg = make_pipeline(LinearRegression())

## Random Forest Regression

In [27]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = make_pipeline(StandardScaler(), RandomForestRegressor())
parameters = {'randomforestregressor__max_depth':[1,2,3,4,5]}
rf_reg = GridSearchCV(rf_reg, parameters)

##Support Vector Regression

In [28]:
from sklearn.gaussian_process.kernels import RBF, ConstantKernel 
from sklearn.svm import SVR

svr = make_pipeline(StandardScaler(), SVR(kernel='linear'))
parameters = {'svr__kernel':['linear', 'rbf'], 'svr__epsilon':[0.1, 0.2, 0.3,0.6], 'svr__C': [0.1,0.5,1,2,3]}
svr = GridSearchCV(svr, parameters)


## Gaussion Process Regression

In [29]:
from sklearn.gaussian_process import GaussianProcessRegressor

gp = make_pipeline(StandardScaler(), GaussianProcessRegressor( n_restarts_optimizer=20, normalize_y=True, copy_X_train=False))
parameters = {'gaussianprocessregressor__kernel':[RBF() + RBF() + RBF()+ RBF()]}
gp = GridSearchCV(gp, parameters)

## Neural Network Univariate

In [30]:
from torch import nn
from skorch import NeuralNetRegressor

class MultivariateLinearRegression(nn.Module):
    def __init__(self, input = 8, output=1, num_units=10, nonlin=nn.LeakyReLU()):
          super(MultivariateLinearRegression, self).__init__()
          self.dense0 = nn.Linear(input, num_units)
          self.nonlin = nonlin
          self.dropout = nn.Dropout(0.1)
          self.output = nn.Linear(num_units, output)
    def forward(self, X):
          X = self.nonlin(self.dense0(X))
          X = self.dropout(X)
          X = self.output(X)
          return X
    def num_parameters(self):
      return sum(p.numel() for p in self.parameters() if p.requires_grad) #only trainable params
      

## single variable 
net = NeuralNetRegressor(
    MultivariateLinearRegression().double(),
    iterator_train__shuffle = False,
    train_split = False,
    verbose = 0)

myNN_reg = make_pipeline(StandardScaler(), net)

params = {
    'neuralnetregressor__lr': [0.001, 0.002, 0.003],
    'neuralnetregressor__max_epochs': [10, 20, 30]
}
myNN_reg = GridSearchCV(myNN_reg, params)



## Neural Network Multivariate

In [31]:
## multivariate variable 
net2 = NeuralNetRegressor(
    MultivariateLinearRegression(output=2).double(),
    iterator_train__shuffle = False,
    train_split = False,
    verbose = 0)

myNN_reg2 = make_pipeline(StandardScaler(), net2)

params = {
    'neuralnetregressor__lr': [0.001, 0.002, 0.003],
    'neuralnetregressor__max_epochs': [10, 20, 30]
}
myNN_reg2 = GridSearchCV(myNN_reg2, params)


## (Optional) Overfit criteria 

In [32]:
# calculate aic for regression
def calculate_aic(n, mse, num_params):
	aic = n * log(mse) + 2 * num_params
	return aic


# calculate bic for regression
def calculate_bic(n, mse, num_params):
	bic = n * log(mse) + num_params * log(n)
	return bic

# Training loop w/ cross-validation 

Below we define a few tools to help with cross validation:

In [33]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from tqdm import tqdm
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)

##function to help store history and metrics of each cross validation run
history = dict()

def append_history(history, algorithm, name, X_test, Y_test, num_parameters):
    #calculate performance metrics
    Y_pred = algorithm.predict(X_test)
    score = algorithm.score(X_test, Y_test)
    #if dict does not contain current alg, create a key
    if name not in history.keys(): 
        history[name] = {"num_parameters": num_parameters, "mse": [],"mae": [], "Rmse":[], "r2": [], "sigmas":[], "predictions": [], "actual": []}
    #store preformance metrics & history
    history[name]["mse"].append(mean_squared_error(Y_test, Y_pred))
    history[name]["Rmse"].append(mean_squared_error(Y_test, Y_pred, squared=False))
    history[name]["mae"].append(mean_absolute_error(Y_test, Y_pred))
    history[name]["r2"].append(score)
    history[name]["predictions"].append(Y_pred)
    history[name]["actual"].append(Y_test)
    if name == "Uni_GP":  
        #MANUALLY SCALE
        scalX_test = algorithm.best_estimator_[0].transform(X_test)
        print(scalX_test.shape)
        _, sigma = algorithm.best_estimator_[1].predict(scalX_test, return_std=True)
        history[name]["sigmas"].append(sigma)
    return history



X_columns  = ["415","445","480", "515", "555", "590", "630", "680"]
# X_columns  = ["mu1","mu2","co1", "co2"]
Y1_columns = ["# emitters λ1 (x1010)"]
Y2_columns = ["# emitters λ2 (x1010)"]
Y_columns  = ["# emitters λ1 (x1010)", "# emitters λ2 (x1010)"]



## Start Training

In [34]:

##For Cross validations 
n_splits = 10
kf = KFold(n_splits=n_splits, random_state=42, shuffle=True) # Define the split - into 10 folds 

##A progress bar because Im fancy
pbar = tqdm(total=100,position=0, leave=True)

##MAIN LOOP
for train_index, test_index in kf.split(df):
    X_train, X_test = df.loc[train_index, X_columns], df.loc[test_index, X_columns]
    Y_train, Y_test = df.loc[train_index, Y2_columns], df.loc[test_index, Y2_columns]
    Y_train_both, Y_test_both = df.loc[train_index, Y_columns], df.loc[test_index, Y_columns]

    ### Gaussian process regression 
    # Fit to data using Maximum Likelihood Estimation of the parameters
    gp.fit(X_train, Y_train.values)
    history = append_history(history, gp, "Uni_GP", X_test, Y_test, num_parameters = len(gp.best_estimator_[1].kernel.hyperparameters) + 1)

    ##NN Regressor multivariate
    myNN_reg2.fit(X_train, Y_train_both.values)
    history = append_history(history, myNN_reg2, "Multi_NN", X_test, Y_test_both, num_parameters = myNN_reg2.best_estimator_[1].module_.num_parameters() + 1)
    ##LINEAR REGRESSION
    lr_reg.fit(X_train, Y_train.values.ravel())
    history = append_history(history, lr_reg, "Uni_LR", X_test, Y_test, num_parameters = len(lr_reg[0].coef_) + 1)

    ##RandomForestRegressor
    rf_reg.fit(X_train, Y_train.values.ravel())
    history = append_history(history, rf_reg, "rf_reg", X_test, Y_test, num_parameters = 1 + 1)

    ##Support Vecotor Regressor 
    svr.fit(X_train, Y_train.values.ravel())
    history = append_history(history, svr, "Uni_SVR", X_test, Y_test, num_parameters = len(svr.best_estimator_[1].coef_) + 1)

    ##NN Regressor
    myNN_reg.fit(X_train, Y_train.values)
    history = append_history(history, myNN_reg, "Uni_NN", X_test, Y_test, num_parameters = myNN_reg.best_estimator_[1].module_.num_parameters()  + 1)

    pbar.update(10)
pbar.close()


  0%|          | 0/100 [00:00<?, ?it/s]

(31, 8)


 10%|█         | 10/100 [02:36<23:32, 15.70s/it]

(31, 8)


 20%|██        | 20/100 [05:08<20:29, 15.37s/it]

...

 80%|████████  | 80/100 [19:59<04:57, 14.89s/it]

(30, 8)


 90%|█████████ | 90/100 [22:22<02:26, 14.69s/it]

(30, 8)


100%|██████████| 100/100 [24:49<00:00, 14.90s/it]


## We can print the results

In [35]:
print('\n')
for i in history.keys():
    print('\n', i,"MSE : ", sum(history[i]["mse"]) / len(history[i]["mse"]))
    print('\n', i,"MAE : ", sum(history[i]["mae"]) / len(history[i]["mae"]))
    print('\n', i,"R^2 : ", sum(history[i]["r2"]) / len(history[i]["r2"]))
    print('\n', i,"RMSE : ", sum(history[i]["Rmse"]) / len(history[i]["Rmse"]))
    print("---")





 Uni_GP MSE :  3.9446007717182683

 Uni_GP MAE :  0.8257662124342693

 Uni_GP R^2 :  0.9949369545406566

 Uni_GP RMSE :  1.8233733019276372
---

 Multi_NN MSE :  20.528850594957387

 Multi_NN MAE :  2.7381712505662197

 Multi_NN R^2 :  0.9718409270632398

 Multi_NN RMSE :  4.365939529076932
---

 Uni_LR MSE :  3.533323019286695

 Uni_LR MAE :  0.9620366903748281

 Uni_LR R^2 :  0.9952320133708717

 Uni_LR RMSE :  1.7216215540283877
---

 rf_reg MSE :  1.052997752431905

 rf_reg MAE :  0.5016122553879946

 rf_reg R^2 :  0.9986093595868946

 rf_reg RMSE :  0.9722747227059239
---

 Uni_SVR MSE :  4.083087092195877

 Uni_SVR MAE :  1.0191091569260262

 Uni_SVR R^2 :  0.9946020805224178

 Uni_SVR RMSE :  1.8294022112965873
---

 Uni_NN MSE :  11.109371265111598

 Uni_NN MAE :  2.308122746087599

 Uni_NN R^2 :  0.9849672467907693

 Uni_NN RMSE :  3.129107206177778
---


## In addition we can plot the results

First we define some tools to plot 

In [36]:
import plotly.graph_objects as graph
def plot(all_history:list, title:str, log = False, error_bars = []):
    """
    input:
        all_history: list of dicts to plot
    ret:
        None: show plotly fig
    """
    symbol_sequence= ['circle-open', 'circle', 'circle-open-dot', 'square']
    fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text=title))) 
    for i in range(len(all_history)):
        fig.add_trace(graph.Scatter(x = all_history[i]["x"], 
                                    y = all_history[i]["y"],
                                    name = all_history[i]["legend"],
                                    mode='markers',
                                    marker_size=5,
                                    marker_symbol=all_history[i]["marker_symbol"],
                                    error_y = dict(type = 'data', # value of error bar given in data coordinates
                                                   array   = error_bars,
                                                   visible = True)))
    if log: fig.update_xaxes(type="log")
    fig.show()

def plotline(all_history:list, title:str, log = False, error_bars = []):
    """
    input:
        all_history: list of dicts to plot
    ret:
        None: show plotly fig
    """
    fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text=title))) 
    for i in range(len(all_history)):
        fig.add_trace(graph.Scatter(x       = all_history[i]["x"], 
                                    y       = all_history[i]["y"],
                                    name    = all_history[i]["legend"])) 
    if log: fig.update_xaxes(type="log")
    fig.show()

## Now we plot each CV validation split predicted vs actual 

In [37]:
import numpy as np
names = list(history.keys())

n_splits = 10

#perfect plot
perfect_plot = {"legend": "actuals", 
                "x": list(np.linspace(0, 100, num=1000, endpoint=True)), 
                "y": list(np.linspace(0, 100, num=1000, endpoint=True)),
                "marker_symbol": 'line-ne-open'}
for alg in names:
  X,Y = [], []           
  for i in range(n_splits):
      X = X + list(history[alg]["actual"][i].values.ravel())
      Y = Y + list(history[alg]["predictions"][i].ravel())

  current_plot = {"legend": "predictions", 
                  "x": X, 
                  "y": Y,
                  "marker_symbol": 'star'}
  
  np.savetxt("X_" + alg + ".csv", X, delimiter=",")
  np.savetxt("Y_" + alg + ".csv", Y, delimiter=",")

  plot([current_plot, perfect_plot], alg + " predicted v actual")



## We can print the best parameters found during cross validation

In [38]:
# num_params = len(gp.best_estimator_[1].coef_) + 1
# print(calculate_bic(len(Y_test), history["linear_regression"]["mse"], num_params))

# print(gp.params)

gp.best_estimator_[1]


GaussianProcessRegressor(copy_X_train=False,
                         kernel=RBF(length_scale=1) + RBF(length_scale=1) + RBF(length_scale=1) + RBF(length_scale=1),
                         n_restarts_optimizer=20, normalize_y=True)